# Decision Tree

In [1]:
import numpy as np
import math

In [2]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparing Dataset

In [9]:
import numpy as np
import math

In [10]:
#below where the file is in gdrive, change with your
data_path = "/content/drive/MyDrive/Colab Notebooks/PRNN_A1/Prnn_datasets/"
dataset = np.loadtxt(data_path + 'PCA_MNIST.csv', delimiter=',',skiprows=1)

In [11]:
dataset.shape

(60000, 11)

In [12]:
# Normalizing dataset
for i in range(1,dataset.shape[1]):
  dataset[:,i] = (dataset[:,i]-dataset[:,i].min())/(dataset[:,i].max()-dataset[:,i].min())

In [15]:
classes = 10
features = 10

In [16]:
train_dataset = dataset[0:40000,:]
test_dataset = dataset[40000:,:]

## Creating decision tree class

Some useful functions are defined before defining the decision tree class, that are used for building decision tree

Here I have assumed the class labels in dataset is present in first column

In [17]:
class node():
  def __init__(self, value,feature,left,right,leaf,depth,cls):
    self.value = value
    self.left = left
    self.right = right
    self.depth = depth
    self.leaf = leaf
    self.feature = feature
    self.cls  = cls 

In [36]:
def entropy(D):
  sum = 0
  x = np.unique(D[:,0])
  for i in range(x.shape[0]):
    d = D[D[:,0]==x[i]]
    p = d.shape[0]/D.shape[0]
    temp = p * math.log(p)
    sum =sum + temp
  sum = -sum
  return(sum)

In [19]:
def gini_index(D):
  sum = 0
  x = np.unique(D[:,0])
  for i in range(x.shape[0]):
    d = D[D[:,0]==x[i]]
    p = d.shape[0]/D.shape[0]
    sum =sum + p*(1-p)
  return(sum)

In [20]:
def maj(D):
  x = np.unique(D[:,0])
  max_cls = 0
  max_count = 0
  for i in range(x.shape[0]):
    d = D[D[:,0]==x[i]]
    if(max_count<d.shape[0]):
      max_count = d.shape[0]
      max_cls = x[i]
  return(max_cls)

In [21]:
def perc_maj_cls(D):
  x = np.unique(D[:,0])
  max_cls = 0
  max_count = 0
  for i in range(x.shape[0]):
    d = D[D[:,0]==x[i]]
    if(max_count<d.shape[0]):
      max_count = d.shape[0]
      max_cls = x[i]
  if(max_count/D.shape[0] >0.95):
    return(max_cls)
  else:
    return(-1)

In [22]:
def grow_trees_entropy(D,depth,k): # k is max depth
  if(depth>=k or D.shape[0]==0):          # user defined depth
    return(None)
  cls = perc_maj_cls(D)
  if(cls!=-1):
    return(node(0,0,None,None,1,depth,cls))
  ent = -1
  featur=0
  value=0
  for features in range(1,D.shape[1]): # first column is ans
    # d = D[:,features]
    # d = np.unique(d)                 # commented code necessary if want to check for each datapoint
    # d = np.sort(d)
    for i in range(1,100):
      d1 = D[D[:,features]<i/100]
      d2 = D[D[:,features]>=i/100]
      p1 = d1.shape[0]/D.shape[0]
      p2 = d2.shape[0]/D.shape[0]
      entropy_split = p1 * entropy(d1)+ p2*entropy(d2)
      if(entropy_split<ent or ent==-1):
        featur = features
        value = i/100
        D1 = d1
        D2 = d2
        ent =entropy_split
  left = grow_trees_entropy(D1,depth+1,k)
  right = grow_trees_entropy(D2,depth+1,k)
  leaf = 0
  if left==None and right == None:
    cls = maj(D)
    leaf = 1
  else:
    cls = -1
  return(node(value,featur,left,right,leaf,depth,cls))

In [26]:
def grow_trees_gini(D,depth,k): # k is max depth
  if(depth>=k or D.shape[0]==0):          # user defined depth
    return(None)
  cls = perc_maj_cls(D)
  if(cls!=-1):
    return(node(0,0,None,None,1,depth,cls))
  ent = -1
  featur=0
  value=0
  for features in range(1,D.shape[1]): # first column is ans
    # d = D[:,features]
    # d = np.unique(d)                 # commented code necessary if want to check for each datapoint
    # d = np.sort(d)
    for i in range(1,100):
      d1 = D[D[:,features]<i/100]
      d2 = D[D[:,features]>=i/100]
      p1 = d1.shape[0]/D.shape[0]
      p2 = d2.shape[0]/D.shape[0]
      gini_split = p1 * gini_index(d1)+ p2*gini_index(d2)
      if(gini_split<ent or ent==-1):
        featur = features
        value = i/100
        D1 = d1
        D2 = d2
        ent =gini_split
  left = grow_trees_gini(D1,depth+1,k)
  right = grow_trees_gini(D2,depth+1,k)
  leaf = 0
  if left==None and right == None:
    cls = maj(D)
    leaf = 1
  else:
    cls = -1
  return(node(value,featur,left,right,leaf,depth,cls))

In [27]:
def run_decision_tree(D,root):
  y=np.zeros(D.shape[0])
  for i in range(D.shape[0]):
    nd =root
    while(1):
      if nd.leaf ==1:
        y[i] = nd.cls
        break
      if(D[i][nd.feature]<nd.value):
        nd = nd.left
      else:
        nd = nd.right
  return(y)

In [28]:
def print_tree(nd):
  print(nd.value,nd.feature,nd.depth,nd.leaf)
  if(nd.left!=None):
    print_tree(nd.left)
  if(nd.right!=None):
    print_tree(nd.right)

In [33]:
class Decision_tree:
  def __init__(self,max_depth=10,gini_flag=0):
    self.max_depth = max_depth
    self.gini_flag=gini_flag
  def train(self,D): # Pass a dataset and max_depth of decision tree as parameter
    if self.gini_flag==0:
      self.root = grow_trees_entropy(D,0,self.max_depth)
    else:
      self.root = grow_trees_gini(D,0,self.max_depth)

  def test(self,D):
    y = run_decision_tree(D,self.root)
    acc = np.sum(y==D[:,0])/D.shape[0]
    return(acc*100)

  def predict(self,D):
    y = run_decision_tree(D,self.root)
    return(y)

## Testing working of Decision tree class on example dataset

In [34]:
tree = Decision_tree()

In [37]:
tree.train(train_dataset)

In [38]:
acc = tree.test(train_dataset)
print("train_acc is ",acc)

train_acc is  89.58749999999999


In [39]:
acc = tree.test(test_dataset)
print("train_acc is ",acc)

train_acc is  90.705


In [40]:
y = tree.predict(test_dataset)